In [1]:
from sklearn.linear_model import Lasso, Ridge, LassoCV, RidgeCV
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import json
import pandas as pd
import numpy as np
import os
import sys
import pickle
import scipy.stats as stats 
import matplotlib.pyplot as plt

# Using the new data cleaning code
First, we have to go from the current directory (inside `old-model-code/experiments/`) back to the base directory.

In [2]:
os.chdir('../../')


In [3]:
os.getcwd()

'/Users/xehu/Desktop/Team Process Mapping/tpm-horse-race-modeling'

In [4]:
stageId_task =  "./data_cache/multi_task_stageId_task_cumulative.pkl"
stageId_cumulative =  "./data_cache/multi_task_stageId_cumulative.pkl"
roundId_task =  "./data_cache/multi_task_roundId_task.pkl"
roundId_cumulative =  "./data_cache/multi_task_roundId_cumulative.pkl"

DATASETFILEPATH = roundId_cumulative

In [5]:
with open(DATASETFILEPATH, "rb") as horseracedataset_file:
    HorseRaceData = pickle.load(horseracedataset_file)

### Key attributes of the HorseRaceDataSet
- HorseRaceData.dvs
- HorseRaceData.composition_features
- HorseRaceData.size_feature
- HorseRaceData.task_features
- HorseRaceData.task_complexity_features
- HorseRaceData.conversation_features

Here's what I wanted before:

`team_composition_features`, `task_features`, `conv_features`, `targets`

In [6]:
targets = HorseRaceData.dvs
team_composition_features = pd.concat([HorseRaceData.size_feature, HorseRaceData.composition_features], axis = 1)
task_features = pd.concat([HorseRaceData.task_features, HorseRaceData.task_complexity_features], axis = 1)
conv_features = HorseRaceData.conversation_features

# Old Cleaning code (for comparison)

In [7]:
def drop_invariant_columns(df):
    """
    Certain features are invariant throughout the training data (e.g., the entire column is 0 throughout).

    These feature obviously won't be very useful predictors, so we drop them.
    
    This function works by identifying columns that only have 1 unique value throughout the entire column,
    and then dropping them.

    @df: the dataframe containing the features (this should be X).
    """
    nunique = df.nunique()
    cols_to_drop = nunique[nunique == 1].index
    return(df.drop(cols_to_drop, axis=1))

In [8]:
def read_and_preprocess_data(path, min_num_chats):
    conv_data  = pd.read_csv(path)

    # Filter this down to teams that have at least min_num of chats
    # Can also comment this out to re-run results on *all* conversations!
    conv_data = conv_data[conv_data["sum_num_messages"] >= min_num_chats]

    # Save the important information

    # DV
    dvs = conv_data[["score","speed","efficiency","raw_duration_min","default_duration_min"]]

    # Team Composition
    composition_colnames = ['birth_year', 'CRT', 'income_max', 'income_min', 'IRCS_GS', 'IRCS_GV', 'IRCS_IB', 'IRCS_IR',
                'IRCS_IV', 'IRCS_RS', 'political_fiscal', 'political_social', 'RME', 'country', 'education_level',
                'gender', 'marital_status', 'political_party', 'race', 'playerCount']
    
    # Select columns that contain the specified keywords
    composition = conv_data[[col for col in conv_data.columns if any(keyword in col for keyword in composition_colnames)]]

    # Task
    task = conv_data[['task', 'complexity']].copy()

    task_map_path = './features/task-mapping/task_map.csv' # get task map
    task_map = pd.read_csv(task_map_path)

    task_name_mapping = {
        "Moral Reasoning": "Moral Reasoning (Disciplinary Action Case)",
        "Wolf Goat Cabbage": "Wolf, goat and cabbage transfer",
        "Guess the Correlation": "Guessing the correlation",
        "Writing Story": "Writing story",
        "Room Assignment": "Room assignment task",
        "Allocating Resources": "Allocating resources to programs",
        "Divergent Association": "Divergent Association Task",
        "Word Construction": "Word construction from a subset of letters",
        "Whac a Mole": "Whac-A-Mole"
    }
    task.loc[:, 'task'] = task['task'].replace(task_name_mapping)
    task = pd.merge(left=task, right=task_map, on = "task", how='left')
    
    # Create dummy columns for 'complexity'
    complexity_dummies = pd.get_dummies(task['complexity'])
    task = pd.concat([task, complexity_dummies], axis=1)   
    task.drop(['complexity', 'task'], axis=1, inplace=True)

    # Conversation
    conversation = conv_data.drop(columns=list(dvs.columns) + list(composition.columns) + ['task', 'complexity', 'roundId', 'gameId', 'message', 'speaker_nickname', 'conversation_num', 'timestamp'])
    conversation = drop_invariant_columns(conversation) # drop invariant conv features

    return composition, task, conversation, dvs

In [9]:
multitask_cumulative_by_stage = 'conv/multi_task_stageId_cumulative_conv.csv'
multitask_cumulative_by_stage_and_task = 'conv/multi_task_stageId_task_cumulative_conv.csv'

In [10]:
# PARAMETERS
desired_target = "score"
data_path = "./data_cache/raw_output/"
min_num_chats = 0


In [11]:
team_composition_features_old, task_features_old, conv_features_old, targets_old = read_and_preprocess_data(data_path + multitask_cumulative_by_stage_and_task, min_num_chats=min_num_chats)

# Number of points in dataset
len(conv_features)

423

## Compare outputs across different data analysis schemas

In [12]:
targets

,score,speed,efficiency,raw_duration_min,default_duration_min,task
0,90.113825,0.000556,0.050063,3.016283,3.0,Divergent Association
1,100.000000,11.074630,1107.462975,4.446283,5.0,Sudoku
2,75.000000,0.000333,0.025000,5.016667,5.0,Moral Reasoning
3,100.000000,0.000333,0.033333,5.014750,5.0,Writing Story
4,100.000000,39.907200,3990.720031,3.004650,5.0,Room Assignment
...,...,...,...,...,...,...
418,100.000000,0.000333,0.033333,5.019383,5.0,Writing Story
419,0.000000,0.000333,0.000000,5.018050,5.0,Wolf Goat Cabbage
420,90.000000,0.000333,0.030000,5.041683,5.0,Moral Reasoning
421,0.000000,0.000333,0.000000,5.021850,5.0,Wolf Goat Cabbage


In [13]:
targets_old

,score,speed,efficiency,raw_duration_min,default_duration_min
0,85.040573,0.000556,0.047245,3.018050,3
1,90.113825,0.000556,0.050063,3.016283,3
2,60.000000,0.000333,0.020000,5.017700,5
3,49.000000,0.000333,0.016333,5.019833,5
4,100.000000,11.074630,1107.462975,4.446283,5
...,...,...,...,...,...
1038,100.000000,0.000333,0.033333,5.017133,5
1039,90.000000,0.000333,0.030000,5.041683,5
1040,90.000000,0.000333,0.030000,5.033550,5
1041,0.000000,0.000333,0.000000,5.021850,5


## Multi-Task Modeling Playground (our old modeling code)

In [14]:
len(conv_features)

423

# Set up X's and y's

In [15]:
X_train = pd.concat([team_composition_features, task_features, conv_features], axis = 1)
y_train = targets

In [16]:
def columns_with_na(df):
    """
    Check and return columns that contain NaN (NA) values in a DataFrame.

    Parameters:
    - df: pandas DataFrame

    Returns:
    - List of column names with NaN values
    """
    # Check for NaN values in each column
    na_columns = df.columns[df.isna().any()].tolist()

    return na_columns

# Check columns that have NA
result = columns_with_na(X_train)
for colname in result:
    print(colname)

In [17]:
X_train

,playerCount,birth_year_mean,birth_year_std,CRT_mean,CRT_std,income_max_mean,income_max_std,income_min_mean,income_min_std,IRCS_GS_mean,...,Q5creativity_input_1,Q25_type6_mixed_motive,High,Low,Medium,PC1,PC2,PC3,PC4,PC5
0,3.0,0.219416,-0.245495,0.905424,-0.529351,1.141173,-1.165105,0.565364,-1.032707,-0.601449,...,1.119660,0,False,True,False,-0.793410,0.059467,0.180201,-0.368505,0.293339
1,3.0,0.219416,-0.245495,0.905424,-0.529351,1.141173,-1.165105,0.565364,-1.032707,-0.601449,...,-1.127059,0,False,True,False,-0.621525,-0.636670,-0.130910,0.151794,0.033753
2,3.0,0.219416,-0.245495,0.905424,-0.529351,1.141173,-1.165105,0.565364,-1.032707,-0.601449,...,-0.261201,0,False,True,False,0.213920,-0.483846,0.695158,-0.479660,0.173264
3,3.0,0.219416,-0.245495,0.905424,-0.529351,1.141173,-1.165105,0.565364,-1.032707,-0.601449,...,1.734098,0,False,True,False,0.928329,-0.673497,0.978094,-0.033090,0.025484
4,3.0,0.219416,-0.245495,0.905424,-0.529351,1.141173,-1.165105,0.565364,-1.032707,-0.601449,...,-0.569070,0,False,True,False,0.352673,-0.668213,0.743449,-0.211188,0.002981
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
418,6.0,0.249246,-0.204565,0.440000,-0.935317,-1.253111,-0.588335,-1.212516,-0.852255,-0.770383,...,1.734098,0,False,True,False,0.007726,1.990084,-1.355174,-0.770918,-1.882589
419,6.0,0.249246,-0.204565,0.440000,-0.935317,-1.253111,-0.588335,-1.212516,-0.852255,-0.770383,...,0.632527,0,False,True,False,1.617761,0.453628,0.568371,0.041172,-1.215361
420,6.0,0.249246,-0.204565,0.440000,-0.935317,-1.253111,-0.588335,-1.212516,-0.852255,-0.770383,...,-0.261201,0,False,False,True,0.200226,1.785060,-0.958813,-0.671809,-1.931370
421,3.0,0.106915,-0.240046,-2.225605,-1.421458,-1.950643,-1.117492,-1.710322,-0.980105,-0.601449,...,0.632527,0,False,True,False,-1.399267,0.393108,2.037213,0.650313,0.973783


## Try LASSO/Ridge Regression, one Set of Features at a Time

Here, we want to implement *leave-one-out cross-validation*, and use Q^2 as our metric.



Two updates to make here:

1. For nested LASSO/Ridge models, add the ability to initialize the model using the previous weights
2. Visualize importance using another library, like SHAP

In [18]:
# Note --- this uses k-fold cross-validation with k = 5 (the default)
# We are testing 10,000 different alphas, so I feel like this is an OK heuristic
def get_optimal_alpha(y_target, feature_columns_list, lasso):

    if(lasso == True):
        model = LassoCV(n_alphas = 10000)
        model.fit(X_train[feature_columns_list], y_train[y_target])
    else:
        model = RidgeCV(n_alphas = 10000)
        model.fit(X_train[feature_columns_list], y_train[y_target])
        
    return model.alpha_ # optimal alpha

In [19]:
def fit_regularized_linear_model(y_target, feature_columns_list, lasso=True, tune_alpha=False, prev_coefs = None, prev_alpha = None):

    if not tune_alpha:
        alpha = 1.0
    if (prev_alpha is not None):
        alpha = prev_alpha # use previous alpha
        print("Setting alpha to previous...")
        print(alpha)
    else:
        # Hyperparameter tune the alpha
        alpha = get_optimal_alpha(y_target, feature_columns_list, lasso=True)

    if lasso:
        model = Lasso(alpha=alpha)
    else:
        model = Ridge(alpha=alpha)

    if(prev_coefs is not None): # set weights to previous coefficients
        print("Setting coefficients ....")
        model.coef_ = prev_coefs

        print(model.coef_)

    # Calculation of Q^2 metric
    squared_model_prediction_errors = []
    squared_average_prediction_errors = []

    # Initialize a list to store coefficients
    coefficients_list = []

    # Leave one out -- iterate through the entire length of the dataset
    for i in range(len(y_train)):
        # Store the evaluation datapoint
        evaluation_X = X_train.iloc[[i]]
        evaluation_y = y_train.iloc[[i]][y_target]

        # Drop the ith datapoint (leave this one out)
        X_train_fold = X_train.drop(X_train.index[i])
        y_train_fold = y_train.drop(y_train.index[i])[y_target]

        # Fit the model
        model.fit(X_train_fold[feature_columns_list], y_train_fold)

        # Save the Prediction Error
        prediction = model.predict(evaluation_X[feature_columns_list])[0]
        squared_model_prediction_errors.append((evaluation_y - prediction) ** 2)

        # Save the Total Error for this fold
        squared_average_prediction_errors.append((evaluation_y - np.mean(y_train_fold)) ** 2)

        # Append the coefficients to the list
        coefficients_list.append(model.coef_)

    # Create a DataFrame with feature names as rows and iteration results as columns
    feature_coefficients = pd.DataFrame(coefficients_list, columns=feature_columns_list).T

    q_squared = 1 - (np.sum(squared_model_prediction_errors) / np.sum(squared_average_prediction_errors))
    print("Q^2: " + str(q_squared))

    return model, q_squared, feature_coefficients


In [20]:
def display_feature_coefficients(feature_coef_df):
    # Initialize a list to store DataFrames for each feature
    dfs = []

    # Iterate through the rows of the input DataFrame
    for feature_name, coefficients in feature_coef_df.iterrows():
        # Calculate the confidence interval without NaN values
        non_nan_coefficients = coefficients[~np.isnan(coefficients)]
        if len(non_nan_coefficients) == 0:
            # Handle the case where there are no valid coefficients
            continue

        mean_coef = non_nan_coefficients.mean()

        # Check if all coefficients in the row are the same
        if len(coefficients.unique()) == 1:
            # If all coefficients are the same, set the lower and upper CI to the mean
            confidence_interval = (mean_coef, mean_coef)
        else:
            std_error = non_nan_coefficients.sem()
            confidence_interval = stats.t.interval(0.95, len(non_nan_coefficients) - 1, loc=mean_coef, scale=std_error)

        # Create a DataFrame for the summary data
        temp_df = pd.DataFrame({
            "Feature": [feature_name],
            "Mean": [mean_coef],
            "Lower_CI": [confidence_interval[0]],
            "Upper_CI": [confidence_interval[1]]
        })

        # Append the temporary DataFrame to the list
        dfs.append(temp_df)

    # Concatenate all the DataFrames in the list into the final summary DataFrame
    summary_df = pd.concat(dfs, ignore_index=True)

    return summary_df

In [21]:
def sort_by_mean_abs(df):
    return df.reindex(df["Mean"].abs().sort_values(ascending=False).index)

In [22]:
# Go through the different types of features and fit models

# First, create a data structure that saves the result
result = {
    "model": [],
    "model_type": [],
    "features_included": [],
    "alpha": [],
    "q_squared": []
}

result_df = pd.DataFrame(result)

## Team composition features

In [23]:
len(team_composition_features.columns)

39

In [24]:
model_ridge_composition, mrc_q2, mrc_feature_coefficients = fit_regularized_linear_model(desired_target, team_composition_features.columns, lasso = False, tune_alpha = True)

result_df = pd.concat([result_df, pd.DataFrame({"model": [model_ridge_composition], "model_type": ["Ridge"], "features_included": ["Team Composition"], "alpha": [model_ridge_composition.alpha.round(4)], "q_squared": [mrc_q2]})], ignore_index=True)

Q^2: -0.05430267118462195


In [25]:
model_ridge_composition

Ridge(alpha=1.653837671654369)

In [26]:
sort_by_mean_abs(display_feature_coefficients(mrc_feature_coefficients))

,Feature,Mean,Lower_CI,Upper_CI
3,CRT_mean,7.180933,7.169756,7.192111
27,country_mean,6.620656,6.604413,6.636898
4,CRT_std,5.040701,5.030868,5.050533
28,country_std,4.970513,4.953992,4.987033
34,marital_status_std,-4.526727,-4.537267,-4.516187
13,IRCS_IB_mean,-4.494894,-4.505121,-4.484666
5,income_max_mean,4.082414,4.071046,4.093782
17,IRCS_IV_mean,3.991824,3.980519,4.003129
24,political_social_std,-3.850109,-3.861533,-3.838684
32,gender_std,-3.822723,-3.832986,-3.812461


In [27]:
model_lasso_composition, mlc_q2, mlc_feature_coefficients = fit_regularized_linear_model(desired_target, team_composition_features.columns, lasso = True, tune_alpha = True)
result_df = pd.concat([result_df, pd.DataFrame({"model": [model_lasso_composition], "model_type": ["Lasso"], "features_included": ["Team Composition"], "alpha": [model_lasso_composition.alpha.round(4)], "q_squared": [mlc_q2]})], ignore_index=True)

Q^2: 0.019274134610606675


In [28]:
model_lasso_composition

Lasso(alpha=1.653837671654369)

## Task Features

In [29]:
len(task_features.columns)

27

In [30]:
model_ridge_task, mrt_q2, mrt_feature_coefficients = fit_regularized_linear_model(desired_target, task_features.columns, lasso = False, tune_alpha = True)
result_df = pd.concat([result_df, pd.DataFrame({"model": [model_ridge_task], "model_type": ["Ridge"], "features_included": ["Task Complexity"], "alpha": [model_ridge_task.alpha.round(4)], "q_squared": [mrt_q2]})], ignore_index=True)

/Users/xehu/anaconda3/envs/tpm_horserace/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.161e+03, tolerance: 3.578e+01
  model = cd_fast.enet_coordinate_descent(


Q^2: 0.19100317383788556


In [31]:
model_ridge_task

Ridge(alpha=0.008166949424698778)

In [32]:
sort_by_mean_abs(display_feature_coefficients(mrt_feature_coefficients))

,Feature,Mean,Lower_CI,Upper_CI
22,Q5creativity_input_1,6.729178,6.723705,6.734650
25,Low,6.156488,6.146279,6.166698
26,Medium,-6.140891,-6.157598,-6.124184
19,Q24eureka_question,-6.025832,-6.033961,-6.017702
6,Q9divisible_unitary,-5.438685,-5.441943,-5.435426
0,Q1concept_behav,-5.011962,-5.015633,-5.008291
4,Q7type_7_battle,4.860173,4.855603,4.864743
13,Q17within_sys_sol,-4.808178,-4.819944,-4.796412
7,Q10maximizing,-4.270455,-4.273326,-4.267583
9,Q13outcome_multip,4.193577,4.189990,4.197164


In [33]:
model_lasso_task, mlt_q2, mlt_feature_coefficients = fit_regularized_linear_model(desired_target, task_features.columns, lasso = True, tune_alpha = True)
result_df = pd.concat([result_df, pd.DataFrame({"model": [model_lasso_task], "model_type": ["Lasso"], "features_included": ["Task Complexity"], "alpha": [model_lasso_task.alpha.round(4)], "q_squared": [mlt_q2]})], ignore_index=True)

/Users/xehu/anaconda3/envs/tpm_horserace/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.161e+03, tolerance: 3.578e+01
  model = cd_fast.enet_coordinate_descent(
/Users/xehu/anaconda3/envs/tpm_horserace/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.131e+03, tolerance: 3.577e+01
  model = cd_fast.enet_coordinate_descent(
/Users/xehu/anaconda3/envs/tpm_horserace/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the

Q^2: 0.19103899721247664


/Users/xehu/anaconda3/envs/tpm_horserace/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.176e+03, tolerance: 3.575e+01
  model = cd_fast.enet_coordinate_descent(
/Users/xehu/anaconda3/envs/tpm_horserace/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.077e+03, tolerance: 3.566e+01
  model = cd_fast.enet_coordinate_descent(
/Users/xehu/anaconda3/envs/tpm_horserace/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the

In [34]:
model_lasso_task

Lasso(alpha=0.008166949424698778)

In [35]:
sort_by_mean_abs(display_feature_coefficients(mlt_feature_coefficients))

,Feature,Mean,Lower_CI,Upper_CI
0,Q1concept_behav,-8.552444,-8.560974,-8.543915
3,Q6type_5_cc,-7.758065,-7.769096,-7.747034
4,Q7type_7_battle,6.535977,6.520004,6.551949
13,Q17within_sys_sol,-6.471123,-6.484387,-6.457859
25,Low,6.161088,6.148212,6.173964
26,Medium,-6.049104,-6.075383,-6.022825
2,Q4type_2_generate,-5.553554,-5.561649,-5.545460
7,Q10maximizing,-4.727330,-4.733857,-4.720803
6,Q9divisible_unitary,-4.616885,-4.623343,-4.610426
19,Q24eureka_question,-4.385410,-4.393583,-4.377236


## Task + Composition Together

In [36]:
# add together weights from previous models
previous_best_weights_ridge = np.array(list(model_ridge_composition.coef_) + list(model_ridge_task.coef_))

In [37]:
task_comp_features = list(task_features.columns) + list(team_composition_features.columns)

model_ridge_taskcomp, mrtc_q2, mrtc_feature_coefficients = fit_regularized_linear_model(desired_target, task_comp_features, lasso = False, tune_alpha = False, prev_coefs = previous_best_weights_ridge, prev_alpha = model_ridge_task.alpha)

Setting alpha to previous...
0.008166949424698778
Setting coefficients ....
[ 1.27669475 -2.25490635 -0.07671827  7.88601901  5.48499032  4.15906451
 -2.19629419  1.66544641  1.13051793 -2.94541788  0.67968398 -2.75169838
  1.37322938 -4.39928908 -2.11071005  0.330777   -2.91369548  3.84922972
 -2.33406651  4.02781083  0.75557631 -2.87099013  1.84042843  1.74850552
 -4.03348857  1.92668128  2.54793209  6.66383225  5.15820054  1.23021809
  2.50279609  2.03206735 -4.22438217  3.2325462  -4.94905519 -0.32160872
  3.83801751 -0.47021067 -0.40503762 -5.00686707 -0.21634678 -4.12748336
 -4.01295145  4.85262332  0.9163254  -5.44557331 -4.28266473  2.86264587
  4.19276895 -2.23116243  0.37543139 -1.16087788 -4.71458023  0.67562457
  1.84745873  0.71423096 -0.88760672 -1.63053972 -6.00646071  0.10237879
 -0.8137943   6.75947543  0.         -0.09947611  6.19566713 -6.09619102]


Q^2: 0.1392494753087452


In [38]:
result_df = pd.concat([result_df, pd.DataFrame({"model": [model_ridge_taskcomp], "model_type": ["Ridge"], "features_included": ["Team Composition + Task Complexity"], "alpha": [model_ridge_taskcomp.alpha.round(4)], "q_squared": [mrtc_q2]})], ignore_index=True)

In [39]:
sort_by_mean_abs(display_feature_coefficients(mrtc_feature_coefficients))

,Feature,Mean,Lower_CI,Upper_CI
30,CRT_mean,6.723211,6.711519,6.734904
54,country_mean,6.712149,6.695456,6.728842
22,Q5creativity_input_1,6.216672,6.210477,6.222867
6,Q9divisible_unitary,-5.532327,-5.535810,-5.528844
29,birth_year_std,5.278908,5.243019,5.314796
...,...,...,...,...
62,political_party_mean,0.325784,0.314556,0.337012
14,Q18ans_recog,0.189427,0.187431,0.191422
53,RME_std,0.066015,0.057486,0.074543
65,race_std,0.038538,0.027509,0.049566


In [40]:
# add together weights from previous models
previous_best_weights_lasso = np.array(list(model_lasso_composition.coef_) + list(model_lasso_task.coef_))

In [41]:
model_lasso_taskcomp, mltc_q2, mltc_feature_coefficients = fit_regularized_linear_model(desired_target, task_comp_features, lasso = True, tune_alpha = False, prev_coefs = previous_best_weights_lasso, prev_alpha = model_lasso_task.alpha)
result_df = pd.concat([result_df, pd.DataFrame({"model": [model_lasso_taskcomp], "model_type": ["Lasso"], "features_included": ["Team Composition + Task Complexity"], "alpha": [model_lasso_taskcomp.alpha.round(4)], "q_squared": [mltc_q2]})], ignore_index=True)

Setting alpha to previous...
0.008166949424698778
Setting coefficients ....
[ 0.06557417 -0.40063576  0.          1.79820354  0.          1.8170176
 -0.          1.74150974  0.         -0.          0.         -0.
 -0.         -0.          0.          0.         -0.         -0.
 -0.          0.2682308   0.         -0.         -0.         -0.
 -0.          0.54556211  0.          0.         -0.          0.
  0.          0.         -0.         -0.         -0.71217176 -0.
 -0.         -0.         -0.         -8.51831239  0.         -5.59639413
 -7.82751374  6.46028787  1.1527539  -4.58321483 -4.74390746  0.
  2.07106766 -0.79266782 -0.         -0.         -6.37297791 -0.
 -0.         -0.          0.         -2.07011228 -4.41688029  0.
 -0.          4.21767909  0.          0.          6.28428783 -5.920582  ]


/Users/xehu/anaconda3/envs/tpm_horserace/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.455e+03, tolerance: 3.577e+01
  model = cd_fast.enet_coordinate_descent(
/Users/xehu/anaconda3/envs/tpm_horserace/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.457e+03, tolerance: 3.574e+01
  model = cd_fast.enet_coordinate_descent(
/Users/xehu/anaconda3/envs/tpm_horserace/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the

Q^2: 0.14043200463978667


/Users/xehu/anaconda3/envs/tpm_horserace/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.219e+03, tolerance: 3.517e+01
  model = cd_fast.enet_coordinate_descent(
/Users/xehu/anaconda3/envs/tpm_horserace/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.417e+03, tolerance: 3.577e+01
  model = cd_fast.enet_coordinate_descent(
/Users/xehu/anaconda3/envs/tpm_horserace/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the

In [42]:
sort_by_mean_abs(display_feature_coefficients(mltc_feature_coefficients))

,Feature,Mean,Lower_CI,Upper_CI
0,Q1concept_behav,-9.862837,-9.874671,-9.851002
3,Q6type_5_cc,-8.109491,-8.124270,-8.094712
13,Q17within_sys_sol,-7.098401,-7.116505,-7.080296
30,CRT_mean,6.624892,6.613209,6.636574
54,country_mean,6.604530,6.587890,6.621170
...,...,...,...,...
16,Q20type_3_type_4,0.000000,0.000000,0.000000
20,Q2intel_manip_1,0.000000,0.000000,0.000000
21,Q21intellective_judg_1,0.000000,0.000000,0.000000
23,Q25_type6_mixed_motive,0.000000,0.000000,0.000000


## Conversation Alone

In [43]:
model_lasso_comms, mlcom_q2, mlcom_feature_coefficients = fit_regularized_linear_model(desired_target, conv_features.columns, lasso = True, tune_alpha = True)
result_df = pd.concat([result_df, pd.DataFrame({"model": [model_lasso_comms], "model_type": ["Lasso"], "features_included": ["Communication"], "alpha": [model_lasso_comms.alpha.round(4)], "q_squared": [mlcom_q2]})], ignore_index=True)

Q^2: -0.009995401758070344


In [44]:
model_lasso_comms

Lasso(alpha=0.6885307815183578)

In [45]:
sort_by_mean_abs(display_feature_coefficients(mlcom_feature_coefficients))

,Feature,Mean,Lower_CI,Upper_CI
2,PC3,-2.820627,-2.829177,-2.812078
3,PC4,-1.975442,-1.986189,-1.964695
4,PC5,0.394311,0.387702,0.400920
1,PC2,0.137456,0.127151,0.147760
0,PC1,0.000364,-0.000306,0.001035


In [46]:
model_lasso_comms

Lasso(alpha=0.6885307815183578)

In [47]:
model_ridge_comms, mrcom_q2, mrcom_feature_coefficients = fit_regularized_linear_model(desired_target, conv_features.columns, lasso = False, tune_alpha = True)
result_df = pd.concat([result_df, pd.DataFrame({"model": [model_ridge_comms], "model_type": ["Ridge"], "features_included": ["Communication"], "alpha": [model_ridge_comms.alpha.round(4)], "q_squared": [mrcom_q2]})], ignore_index=True)

Q^2: -0.011830740179838983


In [48]:
sort_by_mean_abs(display_feature_coefficients(mrcom_feature_coefficients))

,Feature,Mean,Lower_CI,Upper_CI
2,PC3,-3.505312,-3.513764,-3.496860
3,PC4,-2.661512,-2.671839,-2.651184
4,PC5,1.081884,1.074710,1.089057
1,PC2,0.822957,0.812385,0.833529
0,PC1,0.402922,0.396383,0.409461


## Conversation Features + Task Features

In [49]:
task_lasso_weights = np.array(list(model_lasso_task.coef_) + list(np.zeros(len((conv_features.columns)))))

In [50]:
convtask_features = list(task_features.columns) + list(conv_features.columns)
model_lasso_tconv, mltconv_q2, mltconv_feature_coefficients = fit_regularized_linear_model(desired_target, convtask_features, lasso = True, tune_alpha = False, prev_coefs=task_lasso_weights, prev_alpha = model_lasso_task.alpha)
result_df = pd.concat([result_df, pd.DataFrame({"model": [model_lasso_tconv], "model_type": ["Lasso"], "features_included": ["Task Complexity + Communication"], "alpha": [model_lasso_tconv.alpha.round(4)], "q_squared": [mltconv_q2]})], ignore_index=True)

Setting alpha to previous...
0.008166949424698778
Setting coefficients ....
[-8.51831239  0.         -5.59639413 -7.82751374  6.46028787  1.1527539
 -4.58321483 -4.74390746  0.          2.07106766 -0.79266782 -0.
 -0.         -6.37297791 -0.         -0.         -0.          0.
 -2.07011228 -4.41688029  0.         -0.          4.21767909  0.
  0.          6.28428783 -5.920582    0.          0.          0.
  0.          0.        ]


/Users/xehu/anaconda3/envs/tpm_horserace/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.976e+03, tolerance: 3.577e+01
  model = cd_fast.enet_coordinate_descent(
/Users/xehu/anaconda3/envs/tpm_horserace/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.023e+03, tolerance: 3.574e+01
  model = cd_fast.enet_coordinate_descent(
/Users/xehu/anaconda3/envs/tpm_horserace/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the

Q^2: 0.18396324834266198


/Users/xehu/anaconda3/envs/tpm_horserace/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.983e+03, tolerance: 3.574e+01
  model = cd_fast.enet_coordinate_descent(
/Users/xehu/anaconda3/envs/tpm_horserace/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.996e+03, tolerance: 3.577e+01
  model = cd_fast.enet_coordinate_descent(
/Users/xehu/anaconda3/envs/tpm_horserace/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the

In [51]:
sort_by_mean_abs(display_feature_coefficients(mltconv_feature_coefficients))

,Feature,Mean,Lower_CI,Upper_CI
0,Q1concept_behav,-8.512576,-8.521887e+00,-8.503265
3,Q6type_5_cc,-7.612819,-7.624244e+00,-7.601393
13,Q17within_sys_sol,-6.943844,-6.957964e+00,-6.929724
4,Q7type_7_battle,6.370288,6.354007e+00,6.386569
25,Low,5.264221,5.250470e+00,5.277972
2,Q4type_2_generate,-5.214584,-5.223825e+00,-5.205343
26,Medium,-5.057891,-5.084305e+00,-5.031477
6,Q9divisible_unitary,-4.981454,-4.988137e+00,-4.974771
7,Q10maximizing,-4.514068,-4.520699e+00,-4.507437
19,Q24eureka_question,-4.128519,-4.136911e+00,-4.120127


In [52]:
task_ridge_weights = np.array(list(model_ridge_task.coef_) + list(np.zeros(len((conv_features.columns)))))

In [53]:
model_ridge_tconv, mrtconv_q2, mrtconv_feature_coefficients = fit_regularized_linear_model(desired_target, convtask_features, lasso = False, tune_alpha = False, prev_coefs=task_ridge_weights, prev_alpha = model_ridge_task.alpha)
result_df = pd.concat([result_df, pd.DataFrame({"model": [model_ridge_tconv], "model_type": ["Ridge"], "features_included": ["Task Complexity + Communication"], "alpha": [model_ridge_tconv.alpha.round(4)], "q_squared": [mrtconv_q2]})], ignore_index=True)

Setting alpha to previous...
0.008166949424698778
Setting coefficients ....
[-5.00686707 -0.21634678 -4.12748336 -4.01295145  4.85262332  0.9163254
 -5.44557331 -4.28266473  2.86264587  4.19276895 -2.23116243  0.37543139
 -1.16087788 -4.71458023  0.67562457  1.84745873  0.71423096 -0.88760672
 -1.63053972 -6.00646071  0.10237879 -0.8137943   6.75947543  0.
 -0.09947611  6.19566713 -6.09619102  0.          0.          0.
  0.          0.        ]
Q^2: 0.1838973896179339


In [54]:
sort_by_mean_abs(display_feature_coefficients(mrtconv_feature_coefficients))

,Feature,Mean,Lower_CI,Upper_CI
22,Q5creativity_input_1,6.460160,6.454462,6.465857
19,Q24eureka_question,-5.807433,-5.815888,-5.798978
6,Q9divisible_unitary,-5.660397,-5.663703,-5.657090
25,Low,5.227703,5.216981,5.238425
26,Medium,-5.181688,-5.198564,-5.164813
13,Q17within_sys_sol,-5.149586,-5.161920,-5.137251
4,Q7type_7_battle,4.861559,4.856860,4.866257
0,Q1concept_behav,-4.841487,-4.845463,-4.837512
7,Q10maximizing,-4.247274,-4.250179,-4.244370
9,Q13outcome_multip,4.068312,4.064628,4.071995


## Model with All Features

In [55]:
task_composition_lasso_weights = np.array(list(model_lasso_taskcomp.coef_) + list(np.zeros(len((conv_features.columns)))))

In [56]:
all_features = list(task_features.columns) + list(team_composition_features.columns) + list(conv_features.columns)
model_lasso_all, mlall_q2, mlall_feature_coefficients = fit_regularized_linear_model(desired_target, all_features, lasso = True, tune_alpha = False, prev_coefs=task_composition_lasso_weights, prev_alpha = model_lasso_taskcomp.alpha)
result_df = pd.concat([result_df, pd.DataFrame({"model": [model_lasso_all], "model_type": ["Lasso"], "features_included": ["All Features"], "alpha": [model_lasso_all.alpha.round(4)], "q_squared": [mlall_q2]})], ignore_index=True)


Setting alpha to previous...
0.008166949424698778
Setting coefficients ....
[-9.7564718  -0.         -6.61942035 -8.42114469  4.00612627  0.
 -5.22614772 -1.96563551  0.53418034  1.22357119  0.          0.
 -0.         -6.68940042  0.          0.          0.         -0.05060947
 -3.53298078 -2.64051779 -0.         -0.          3.80692688  0.
 -0.          6.13482982 -0.24652387  2.03985992  4.30217251  6.43533075
  7.62801893  4.89111971  3.9126325  -1.62596169  1.52119256 -0.70016116
 -1.91887978  0.42261081 -2.18286753  1.2376873  -3.81935636 -2.74403442
 -0.7320332  -3.12263766  2.49428714 -2.36416427  3.18560846 -0.37562281
 -2.70259923  2.30440405  2.37434033 -4.37133072  0.06803866 -0.03262011
  6.63509818  5.23110271 -0.92282788  1.18162303  2.23459029 -4.2286681
  1.56699356 -4.97015739  0.3711275   2.23242099  0.48284847 -0.14533561
  0.          0.          0.          0.          0.        ]


/Users/xehu/anaconda3/envs/tpm_horserace/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.345e+02, tolerance: 3.577e+01
  model = cd_fast.enet_coordinate_descent(
/Users/xehu/anaconda3/envs/tpm_horserace/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.338e+02, tolerance: 3.574e+01
  model = cd_fast.enet_coordinate_descent(
/Users/xehu/anaconda3/envs/tpm_horserace/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the

Q^2: 0.14440925965471485


/Users/xehu/anaconda3/envs/tpm_horserace/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.490e+02, tolerance: 3.577e+01
  model = cd_fast.enet_coordinate_descent(
/Users/xehu/anaconda3/envs/tpm_horserace/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.270e+02, tolerance: 3.517e+01
  model = cd_fast.enet_coordinate_descent(
/Users/xehu/anaconda3/envs/tpm_horserace/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the

In [57]:
sort_by_mean_abs(display_feature_coefficients(mlall_feature_coefficients))

,Feature,Mean,Lower_CI,Upper_CI
0,Q1concept_behav,-8.949457,-8.966161,-8.932753
30,CRT_mean,7.339967,7.329126,7.350809
3,Q6type_5_cc,-7.003571,-7.018617,-6.988526
2,Q4type_2_generate,-6.193135,-6.210164,-6.176106
54,country_mean,6.187083,6.170483,6.203684
...,...,...,...,...
1,Q3type_1_planning,0.000000,0.000000,0.000000
16,Q20type_3_type_4,0.000000,0.000000,0.000000
20,Q2intel_manip_1,0.000000,0.000000,0.000000
21,Q21intellective_judg_1,0.000000,0.000000,0.000000


In [58]:
task_composition_ridge_weights = np.array(list(model_ridge_taskcomp.coef_) + list(np.zeros(len((conv_features.columns)))))

model_ridge_all, mrall_q2, mrall_feature_coefficients = fit_regularized_linear_model(desired_target, all_features, lasso = False, tune_alpha = True, prev_coefs=task_composition_ridge_weights, prev_alpha = model_ridge_taskcomp.alpha)
result_df = pd.concat([result_df, pd.DataFrame({"model": [model_ridge_all], "model_type": ["Ridge"], "features_included": ["All Features"], "alpha": [model_ridge_all.alpha.round(4)], "q_squared": [mrall_q2]})], ignore_index=True)

Setting alpha to previous...
0.008166949424698778
Setting coefficients ....
[-4.99731748  0.69386593 -4.56009482 -3.4736652   3.88501026 -0.59631965
 -5.55965273 -3.91455768  2.99565489  3.85776083 -1.59037976  0.31943126
 -1.04063996 -4.35647999  0.19716454  1.93062397  1.00318775 -2.00361014
 -2.13935397 -4.54921065 -0.28847008 -0.56424332  6.3358406   0.
 -1.97641361  4.2312226  -2.25480899  2.02772383  5.46111054  7.55662253
  7.73656383  4.98484222  3.9567871  -1.62826322  1.57086997 -0.76021641
 -1.92157753  0.43862012 -2.22833901  1.23734007 -3.85763963 -2.7745606
 -0.78269851 -3.17089951  2.56987943 -2.36990195  3.24521009 -0.3841056
 -2.79374904  2.36964649  2.49651806 -4.47819004  0.10366642 -0.04480123
  6.74905325  5.34952931 -0.94469127  1.26987195  2.27651598 -4.31428315
  1.67536542 -5.06001092  0.38519087  2.31814317  0.51198529 -0.19528055
  0.          0.          0.          0.          0.        ]


Q^2: 0.14186724634008407


In [59]:
sort_by_mean_abs(display_feature_coefficients(mrall_feature_coefficients))

,Feature,Mean,Lower_CI,Upper_CI
30,CRT_mean,7.457610,7.446683,7.468537
54,country_mean,6.293636,6.276990,6.310283
22,Q5creativity_input_1,5.925478,5.919124,5.931833
6,Q9divisible_unitary,-5.772092,-5.775742,-5.768443
51,political_social_std,-4.937775,-4.948223,-4.927327
...,...,...,...,...
14,Q18ans_recog,0.214036,0.212005,0.216066
28,birth_year_mean,-0.163221,-0.204312,-0.122130
52,RME_mean,-0.118523,-0.128264,-0.108782
35,income_min_std,-0.037050,-0.047334,-0.026766


# Dataframe that summarizes all these experiments!

In [60]:
result_df.sort_values(by = "q_squared", ascending = False)

,model,model_type,features_included,alpha,q_squared
3,Lasso(alpha=0.008166949424698778),Lasso,Task Complexity,0.0082,0.191039
2,Ridge(alpha=0.008166949424698778),Ridge,Task Complexity,0.0082,0.191003
8,Lasso(alpha=0.008166949424698778),Lasso,Task Complexity + Communication,0.0082,0.183963
9,Ridge(alpha=0.008166949424698778),Ridge,Task Complexity + Communication,0.0082,0.183897
10,Lasso(alpha=0.008166949424698778),Lasso,All Features,0.0082,0.144409
11,Ridge(alpha=0.008166949424698778),Ridge,All Features,0.0082,0.141867
5,Lasso(alpha=0.008166949424698778),Lasso,Team Composition + Task Complexity,0.0082,0.140432
4,Ridge(alpha=0.008166949424698778),Ridge,Team Composition + Task Complexity,0.0082,0.139249
1,Lasso(alpha=1.653837671654369),Lasso,Team Composition,1.6538,0.019274
6,Lasso(alpha=0.6885307815183578),Lasso,Communication,0.6885,-0.009995


In [61]:
fit_regularized_linear_model(desired_target, ["Low", "Medium", "High"], lasso = False, tune_alpha = True)

Q^2: 0.04275837752271994


(Ridge(alpha=0.031890271728889065),
 0.04275837752271994,
               0          1          2          3          4          5    \
 Low      9.746128   9.721357   9.783998   9.721357   9.721357   9.803828   
 Medium -11.422042 -11.409659 -11.440972 -11.409659 -11.409659 -11.391715   
 High     1.675914   1.688302   1.656975   1.688302   1.688302   1.587888   
 
               6          7          8          9    ...        413        414  \
 Low      9.808609   9.780019   9.721357   9.690735  ...   9.839541   9.812756   
 Medium -11.386937 -11.415509 -11.409659 -11.504738  ... -11.356024 -11.382792   
 High     1.578328   1.635491   1.688302   1.814003  ...   1.516483   1.570036   
 
               415        416        417        418        419        420  \
 Low      9.827269   9.669902   9.726368   9.721357   9.971921   9.989321   
 Medium -11.368289 -11.525558 -11.412164 -11.409659 -11.534912 -11.881039   
 High     1.541020   1.855655   1.685796   1.688302   1.562992   1.8917

In [62]:
fit_regularized_linear_model(desired_target, ["playerCount"], lasso = True, tune_alpha = True)

Q^2: -0.002229321060304601


(Lasso(alpha=2.088032120973947),
 -0.002229321060304601,
                   0         1    2         3         4         5         6    \
 playerCount  0.022136  0.039123  0.0  0.039123  0.039123  0.018504  0.021264   
 
                   7         8    9    ...       413       414       415  416  \
 playerCount  0.004758  0.039123  0.0  ...  0.039123  0.023659  0.032037  0.0   
 
              417  418       419  420  421       422  
 playerCount  0.0  0.0  0.117511  0.0  0.0  0.004758  
 
 [1 rows x 423 columns])

In [63]:
# model to look at composition without playercount
fit_regularized_linear_model(desired_target, list(set(team_composition_features.columns)-set(["playerCount"])), lasso = True, tune_alpha = True)


Q^2: 0.022601441136104627


(Lasso(alpha=1.6221538443486787),
 0.022601441136104627,
                             0         1         2         3         4    \
 political_fiscal_mean -0.000000 -0.000000 -0.000000 -0.000000 -0.000000   
 gender_std            -0.000000 -0.000000 -0.000000 -0.000000 -0.000000   
 income_min_mean        1.760515  1.786452  1.720869  1.786452  1.786452   
 political_party_mean  -0.000000 -0.000000 -0.000000 -0.000000 -0.000000   
 IRCS_IB_std            0.000000  0.000000  0.000000  0.000000  0.000000   
 marital_status_mean   -0.000000 -0.000000 -0.000000 -0.000000 -0.000000   
 IRCS_GS_mean          -0.000000 -0.000000 -0.000000 -0.000000 -0.000000   
 IRCS_IV_mean          -0.000000 -0.000000 -0.000000 -0.000000 -0.000000   
 income_max_mean        1.795129  1.759081  1.850236  1.759081  1.759081   
 IRCS_IR_std           -0.000000 -0.000000 -0.000000 -0.000000 -0.000000   
 RME_std                0.000000  0.000000  0.000000  0.000000  0.000000   
 political_party_std   -0.00000

# Feature Importance

In [64]:
sort_by_mean_abs(display_feature_coefficients(mlall_feature_coefficients))

,Feature,Mean,Lower_CI,Upper_CI
0,Q1concept_behav,-8.949457,-8.966161,-8.932753
30,CRT_mean,7.339967,7.329126,7.350809
3,Q6type_5_cc,-7.003571,-7.018617,-6.988526
2,Q4type_2_generate,-6.193135,-6.210164,-6.176106
54,country_mean,6.187083,6.170483,6.203684
...,...,...,...,...
1,Q3type_1_planning,0.000000,0.000000,0.000000
16,Q20type_3_type_4,0.000000,0.000000,0.000000
20,Q2intel_manip_1,0.000000,0.000000,0.000000
21,Q21intellective_judg_1,0.000000,0.000000,0.000000


In [65]:
def plot_top_n_features(data, n, filepath):
    # Calculate the absolute mean value and sort the DataFrame in descending order
    data['Absolute_Mean'] = data['Mean'].abs()
    top_n_features = data.sort_values(by='Absolute_Mean', ascending=False).head(n)

    # Define color mapping for the features
    color_map = {}
    name_map = {}
    for feature in task_features.columns:
        color_map[feature] = 'yellowgreen'
        name_map[feature] = "Task Feature"
    for feature in conv_features.columns:
        color_map[feature] = 'powderblue'
        name_map[feature] = "Conversation Feature"
    for feature in team_composition_features.columns:
        color_map[feature] = 'lightpink'
        name_map[feature] = "Team Composition Feature"

    # Create a horizontal bar graph
    plt.figure(figsize=(10, 6))

    handles = []

    for feature in top_n_features['Feature']:
        color = color_map.get(feature, 'k')  # Default to black if not in any list
        bars = plt.barh(feature, top_n_features[top_n_features['Feature'] == feature]['Mean'], color=color)
        handles.append(bars[0])

    # Customize the plot
    plt.xlabel('Mean Coefficient (Across LOO Cross Validation)', fontsize = 14)
    plt.title(f'Top {n} features for {desired_target} (min chats = {min_num_chats})', fontsize=20)
    plt.gca().invert_yaxis()  # Invert the y-axis to display the highest value at the top

    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)

    # Create a legend outside the plot area with unique labels
    unique_features = []
    unique_labels = []
    for feature in top_n_features['Feature']:
        if name_map.get(feature, feature) not in unique_labels:
            unique_labels.append(name_map.get(feature, feature))
            unique_features.append(feature)

    legend_handles = [plt.Line2D([0], [0], color=color_map.get(feature, 'k'), lw=4, label=name_map.get(feature, feature)) for feature in unique_features]
    plt.legend(handles=legend_handles, loc='center left', fontsize = 14, bbox_to_anchor=(1, 0.5))

    # Add labels to the bars with increased text size and Mean rounded to 2 decimals, consistently inside the bar
    label_offset = 0.4  # Adjust this value for proper spacing
    for bar, value, feature in zip(handles, top_n_features['Mean'], top_n_features['Feature']):
        label_x = (max(value, 0) if value >= 0 else min(value, 0))
        bbox = bar.get_bbox()
        label_y = bbox.bounds[1] + label_offset
        if value >= 0:
            plt.text(label_x, label_y, f'{value:.2f}', va='center', fontsize=12)
        else:
            plt.text(label_x, label_y, f'{value:.2f}', ha='right', va='center', fontsize=12)

    # Show the plot
    plt.savefig(filepath + ".svg")
    plt.savefig(filepath + ".png")
    plt.show()

Questions:
- More deeply understand difference between LASSO and Ridge
- Better understand `alpha` hyperparameter
- Why doesn't more features mean a better R^2? (Wouldn't the model 'throw out' features that don't work?)